In [10]:
!pip3 install numpy --upgrade
!pip3 install scipy --upgrade

In [11]:
import fft_project
import psutil
import logging
import time
import scipy
import numpy as np
from importlib import reload
from fft_project import base


In [12]:
'''Below are the testcases, where we controlled the multithreading and creating copies
of our version of 2DFFT versus Scipy fft2'''

#Printing out Current CPU Info
print("="*40, "CPU Info", "="*40)
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 2800.00Mhz
Min Frequency: 2800.00Mhz
Current Frequency: 2800.00Mhz


In [13]:
def random_matrix(n, m):
    logging.info(f'Matrix shape: {n}x{m}')
    result = np.zeros((n, m), dtype=np.complex128)
    result.real = np.random.rand(n, m)
    return result

reload(logging)
base.prepare_logger()


Multithreading enabled and type conversions

In [14]:
import numpy as np
import scipy.fft
import logging
from scipy.fft import fft2, ifft2
import fft_project

def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    timing = %timeit -o scipy.fft.fft2(result, workers= -1)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(result, return_copy=True, use_threads=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')

    expected = fft2(result, workers= -1)
    result = fft_project.fft2d(result, return_copy=True, use_threads=True)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=True)
    timing = %timeit -o scipy.fft.ifft2(transformed, workers= -1)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(transformed, return_copy=True, use_threads=True, inverse=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')

    inversed_scipy = ifft2(transformed, workers= -1)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=True, use_threads=True, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192


454 ms ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.1 s ± 92.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 454 ms ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.1 s ± 92.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


559 ms ± 44.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.05 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 559 ms ± 44.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.05 s ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


With multithreading enabled, no type conversions

In [15]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    timing = %timeit -o scipy.fft.fft2(result, workers =-1)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(result, return_copy=False, use_threads=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')

    expected = fft2(result, workers= -1)
    result = fft_project.fft2d(result, return_copy=False, use_threads=True)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=True)
    timing = %timeit -o scipy.fft.ifft2(transformed, workers= -1)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(transformed, return_copy=False, use_threads=True, inverse=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')

    inversed_scipy = ifft2(transformed, workers= -1)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=False, use_threads=True, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()


[INFO]  Matrix shape: 4096x8192


469 ms ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 592 ms ± 47.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 469 ms ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


592 ms ± 47.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


511 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 608 ms ± 9.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 511 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


608 ms ± 9.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


No multithreading, with type conversions

In [16]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    timing = %timeit -o scipy.fft.fft2(result)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(result, return_copy=True, use_threads=False)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')
    expected = fft2(result)
    result = fft_project.fft2d(result, return_copy=True, use_threads=False)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=True, use_threads=False)
    timing = %timeit -o scipy.fft.ifft2(transformed)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(transformed, return_copy=True, use_threads=False, inverse=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')

    inversed_scipy = ifft2(transformed)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=True, use_threads=False, inverse=True
    )
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192


1.64 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.45 s ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 1.64 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.45 s ± 9.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


1.6 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.46 s ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 1.6 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.46 s ± 30.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True


No multithreading, no type conversions

In [17]:
def fft_cpp_impl_test():
    result = random_matrix(4096, 4096 * 2)
    timing = %timeit -o scipy.fft.fft2(result)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(result, return_copy=False, use_threads=False)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')
    expected = fft2(result)
    result = fft_project.fft2d(result, return_copy=False, use_threads=False)
    correctness = np.allclose(result, expected, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

def inverse_test():
    a = random_matrix(4096 * 2, 4096)
    transformed = fft_project.fft2d(a, return_copy=False, use_threads=False)
    timing = %timeit -o scipy.fft.ifft2(transformed)
    scipy_time = timing
    timing = %timeit -o fft_project.fft2d(transformed, return_copy=False, use_threads=False, inverse=True)
    our_time = timing
    logging.info(f'Our: {our_time}')
    logging.info(f'Scipy: {scipy_time}')
    inversed_scipy = ifft2(transformed)
    inversed_our = fft_project.fft2d(
    transformed, return_copy=False, use_threads=False, inverse=True)
    correctness = np.allclose(inversed_our, inversed_scipy, atol=0.05)
    logging.info(f'Is the result correct: {correctness}')
    assert correctness

fft_cpp_impl_test()
inverse_test()

[INFO]  Matrix shape: 4096x8192


1.64 s ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.11 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 1.64 s ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.11 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
[INFO]  Matrix shape: 8192x4096


1.74 s ± 80.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Our: 1.01 s ± 19.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
[INFO]  Scipy: 1.74 s ± 80.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


1.01 s ± 19.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[INFO]  Is the result correct: True
